# Single Query

Before jumping into anything more complex, I want to get a sense of how well a frontier model performs with a single prompt—without any data cleaning or preprocessing. This will also serve as a benchmark for future approaches.

With this approach, the prompt will be extremely important, so as part of this exploration, I’ll experiment with a few different prompts.

To keep things simple, I’m going to ignore the demographics/segment columns and focus only on individual response columns. I’ll also highlight the challenges this approach will face when processing large datasets, as well as issues related to data cleaning, such as removing PII, profanity, etc.

Ideas for improving prompts largely come from:
https://platform.openai.com/docs/guides/prompt-engineering

In [1]:
import os
import json
import polars as pl
from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path
from IPython.display import Markdown

load_dotenv(Path(".env"))
client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
)

In [2]:
df = pl.read_csv(Path("../wallmart.csv"))
df.head()

Session ID,Date started,"Imagine you are writing a complaint letter to the head of Walmart...in detail, tell us the things you would complain about and why they matter?",What do you think Target does better than Walmart?,What do you LOVE most about Walmart?,Gender,Age group,State,Who does it better? Target or Walmart?
str,str,str,str,str,str,str,str,str
"""94f2d4c3-b513-411c-b505-a11290…","""10/02/2022 01:03""","""The customer service that your…","""Their overall approach to cust…","""There's nothing I LOVE about W…","""Male""","""45-54""","""Texas""","""Target"""
"""1797c6f2-c501-44b7-b549-a33c29…","""10/02/2022 01:03""","""I would complain about how mor…",null,"""I love the diversity of produc…","""Female""","""18-24""","""Texas""","""Walmart"""
"""20e1a746-8311-450b-ba54-62d15f…","""10/02/2022 01:03""","""Why do you quit taking cash at…","""Target still has a snack bar u…","""The employees. They try when …","""Male""","""55-64""","""Texas""","""Target"""
"""234eb679-ffb8-4e5e-a45a-e50a7c…","""10/02/2022 01:03""","""You don't just go into Walmart…",null,"""In the small town that I live …","""Male""","""35-44""","""Texas""","""Walmart"""
"""511293b3-42e1-4712-a3a9-8838e0…","""10/02/2022 01:03""","""East Waco needs a Walmart neig…",null,"""The low price""","""Male""","""25-34""","""Texas""","""Walmart"""


In [3]:
def clean_responses(responses: pl.Series) -> pl.Series:
    """Clean a series of text responses by removing whitespace, empty strings, and null values.

    Args:
        responses: A polars Series containing text responses

    Returns:
        A cleaned polars Series with whitespace stripped, empty strings removed,
        newlines replaced with spaces, multiple spaces condensed, and null values dropped
    """
    # Clean up whitespace
    cleaned_whitespace = responses.str.strip_chars().str.replace_all(r"\s+", " ")
    # Drop all nulls and empty strings
    return cleaned_whitespace.filter(
        cleaned_whitespace.str.len_chars() > 0
    ).drop_nulls()

In [4]:
# For now just focus on a single column
question = "Imagine you are writing a complaint letter to the head of Walmart...in detail, tell us the things you would complain about and why they matter?"
cleaned_responses = clean_responses(df[question])
formatted_responses = "\n".join(cleaned_responses)

## Prompt 1: Basic instructions

In [5]:
prompt = f"""
You are an experienced market researcher. You are given a question and a list of responses. Your job is to extract themes from the responses.

For each theme, come up with a name that captures the theme, and a summary, that gives more context on the theme.
It is important that the themes are not overlapping, and that they are not too general or too specific.

Question: {question}

Responses:
{formatted_responses}
"""
answer = client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[{"role": "user", "content": prompt}],
).choices[0].message.content

display(Markdown(answer))

### Theme 1: Customer Service Concerns
**Summary:** Many respondents expressed dissatisfaction with the customer service at Walmart, citing rude or unhelpful employees, lack of assistance, and poor training. This theme highlights the importance of customer service in ensuring a positive shopping experience and customer retention.

### Theme 2: Checkout Process Frustrations
**Summary:** A significant number of complaints focused on the checkout process, particularly the lack of open registers and the over-reliance on self-checkout lanes. Customers feel inconvenienced by long lines and the expectation to self-checkout, especially when there are few cashiers available.

### Theme 3: Store Cleanliness and Organization
**Summary:** Respondents frequently mentioned issues with store cleanliness and organization. Complaints included dirty bathrooms, cluttered aisles, and disorganized shelves, which make shopping difficult and unpleasant.

### Theme 4: Product Availability and Quality
**Summary:** Many customers are frustrated with the frequent out-of-stock items and the quality of certain products, particularly in the produce and meat sections. This theme underscores the need for better inventory management and quality control to meet customer expectations.

### Theme 5: Employee Treatment and Staffing
**Summary:** Concerns were raised about the treatment of employees and the impact of understaffing on service quality. Respondents noted that employees often seem overworked and underpaid, which affects their ability to provide good service.

### Theme 6: Pricing and Value Perception
**Summary:** Some customers expressed dissatisfaction with pricing, feeling that Walmart's prices are not as competitive as they used to be. There is also a desire for more transparency and consistency in pricing, as well as better deals and promotions.

### Theme 7: Store Hours and Accessibility
**Summary:** A few respondents mentioned the inconvenience of reduced store hours and the lack of 24-hour availability. This theme highlights the importance of accessibility and convenience in meeting customer needs.

### Theme 8: Racial Profiling and Discrimination
**Summary:** A serious concern raised by a respondent involved racial profiling and discrimination experienced in the store. This theme emphasizes the need for Walmart to address and prevent discriminatory practices to ensure a welcoming environment for all customers.

## Prompt 2: Research Terminology and more specific instructions

In [ ]:
prompt = f"""
You are a qualitative research analyst specializing in Thematic Analysis. Your task is to analyze open-ended customer feedback and systematically extract themes using qualitative coding techniques.

## **Task**
Conduct a thematic analysis of the provided responses by:
1. Identifying recurring patterns across responses. Only themes that appear multiple times should be included.
2. Assigning a concise, descriptive theme label that represents the core idea.
3. Writing a detailed summary that provides additional context beyond the label, highlighting key aspects and nuances that the name alone may not fully capture.

## **Methodological Guidelines**
- Ensure mutual exclusivity: themes should not significantly overlap in meaning.
- Theme names should be short and precise, while summaries should explain the broader context, implications, and nuances.
- Maintain actionability: the summary should articulate why the theme is relevant and its impact on customer experience.

## **Output Format**
Each theme should follow this structure:

- Theme Name: (Concise label representing the central idea)
- Summary: (Detailed description providing context, key aspects, and nuances)

## **Question Context**
{question}

## **Dataset**
{formatted_responses}
"""
answer = client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[{"role": "user", "content": prompt}],
).choices[0].message.content

display(Markdown(answer))

### Theme Name: Insufficient Checkout Staffing
- **Summary:** A significant number of customers express frustration over the lack of open checkout lanes, both staffed and self-service, leading to long wait times. This issue is exacerbated during peak shopping hours and holidays, causing inconvenience and dissatisfaction. Customers feel that the store should employ more cashiers to improve the checkout experience, especially for those who prefer not to use self-checkout or have large quantities of items. The lack of open lanes is seen as a failure to provide efficient service, impacting the overall shopping experience negatively.

### Theme Name: Poor Customer Service
- **Summary:** Many customers report negative experiences with Walmart's customer service, citing rude and unhelpful staff. This includes difficulty in finding assistance when needed, dismissive attitudes, and a lack of initiative to help customers. Such experiences lead to dissatisfaction and deter customers from returning. Improving customer service is crucial for enhancing customer satisfaction and loyalty, as it directly affects the perception of the store and the likelihood of repeat business.

### Theme Name: Store Cleanliness and Organization
- **Summary:** Customers frequently mention issues with the cleanliness and organization of Walmart stores. Complaints include dirty bathrooms, cluttered aisles, and disorganized shelves, which make it difficult to find items. Cleanliness is particularly emphasized due to health concerns, especially in the context of the pandemic. A well-maintained and organized store environment is essential for a positive shopping experience, as it affects both the ease of shopping and the store's overall image.

### Theme Name: Product Availability and Stock Issues
- **Summary:** There are numerous complaints about items being out of stock or not available, which frustrates customers who rely on Walmart for their shopping needs. This issue is compounded by discrepancies between online and in-store stock levels. Ensuring consistent product availability is vital for customer satisfaction, as it directly impacts the ability to complete shopping trips successfully and can drive customers to seek alternatives elsewhere.

### Theme Name: Self-Checkout Concerns
- **Summary:** While some customers appreciate the convenience of self-checkout, others express dissatisfaction with being forced to use it due to a lack of staffed lanes. Concerns include technical issues with machines, the absence of discounts for self-service, and the inconvenience for those who prefer human interaction or have large purchases. Addressing these concerns by balancing self-checkout with staffed lanes and ensuring functional equipment can improve the checkout experience.

### Theme Name: Employee Treatment and Morale
- **Summary:** Customers express concern over the treatment of Walmart employees, noting that poor morale and inadequate staffing levels contribute to negative customer service experiences. There is a call for better wages and working conditions to improve employee satisfaction, which in turn could enhance customer interactions. Addressing employee welfare is seen as a way to improve service quality and the overall shopping environment.

### Theme Name: Pricing and Value Perception
- **Summary:** Some customers feel that Walmart's pricing is not as competitive as it once was, particularly for everyday items and produce. There is a perception that prices have increased without a corresponding increase in quality or service. Ensuring competitive pricing and clear value propositions is important for maintaining Walmart's reputation as a cost-effective shopping destination and retaining price-sensitive customers.

## Prompt 3: Examples / Multi-shot

In [ ]:
prompt = f"""
You are a qualitative research analyst specializing in Thematic Analysis. Your task is to analyze open-ended customer feedback and systematically extract themes using qualitative coding techniques.

## **Task**
Conduct a thematic analysis of the provided responses by:
1. Identifying recurring patterns across responses. Only themes that appear multiple times should be included.
2. Assigning a concise, descriptive theme label that represents the core idea.
3. Writing a detailed summary that provides additional context beyond the label, highlighting key aspects and nuances that the name alone may not fully capture.

## **Methodological Guidelines**
- Ensure mutual exclusivity: themes should not significantly overlap in meaning.
- Theme names should be short and precise, while summaries should explain the broader context, implications, and nuances.
- Maintain actionability: the summary should articulate why the theme is relevant and its impact on customer experience.

## **Output Format**
Each theme should follow this structure:

- Theme Name: (Concise label representing the central idea)
- Summary: (Detailed description providing context, key aspects, and nuances)

## **Examples**

**Example 1**: Good theme

- Theme Name: Long Checkout Lines
- Summary: A significant number of responses highlight the frustration with long checkout lines due to insufficient staffing. Customers express dissatisfaction with the limited number of open registers, especially during peak hours, leading to extended wait times. This issue is exacerbated by the reliance on self-checkout lanes, which many customers find inconvenient or unsuitable, particularly for those with large purchases or those who prefer human interaction. The lack of cashiers not only affects the speed of service but also the overall shopping experience, as customers feel neglected and undervalued.

This theme is specific and actionable. It highlights a clear operational issue—insufficient staffing at checkout lanes, particularly during busy times. The summary provides enough context to suggest possible solutions (e.g., opening more registers, improving staffing) to address the pain point, making it actionable for process improvements.

**Example 2**: Bad theme

- Theme Name: Customer Service Issues
- Summary: Many responses highlight dissatisfaction with Walmart's customer service, noting poor service quality, lack of empathy, and the prioritization of profit over customer satisfaction. This theme reflects a significant pain point for customers, affecting their overall experience and perception of the brand.

This theme is too general and vague to be actionable. While customer service dissatisfaction is a valid concern, the theme lacks specificity in identifying which aspects of service need improvement (e.g., wait times, employee behavior, training). It also doesn't suggest concrete solutions or areas for operational change. A more focused theme like "Long Checkout Lines" provides a specific, actionable area for improvement, whereas "Customer Service Issues" fails to do so.

## **Question Context**
{question}

## **Dataset**
{formatted_responses}
"""
answer = client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[{"role": "user", "content": prompt}],
).choices[0].message.content

display(Markdown(answer))

Based on the provided dataset of customer feedback, I have conducted a thematic analysis and identified several recurring themes. Here are the themes along with their summaries:

- **Theme Name: Insufficient Checkout Staffing**
  - **Summary:** A prevalent concern among customers is the lack of open checkout lanes, leading to long wait times. Many express frustration with the reliance on self-checkout systems, which are often crowded and not preferred by all customers, especially those with large purchases or those who are not comfortable with technology. The limited number of staffed registers, particularly during peak hours, exacerbates this issue, making the checkout process a significant pain point. Customers suggest that increasing the number of open registers and hiring more cashiers could improve the shopping experience by reducing wait times and providing a more personalized service.

- **Theme Name: Poor Customer Service**
  - **Summary:** Numerous responses highlight dissatisfaction with the quality of customer service. Customers report experiences with rude or unhelpful staff, difficulty in finding assistance when needed, and a general lack of employee engagement. This theme underscores the importance of customer service training and the need for employees to be more attentive and approachable. Improving customer service is crucial for enhancing customer satisfaction and encouraging repeat visits.

- **Theme Name: Store Cleanliness and Organization**
  - **Summary:** Customers frequently mention issues with store cleanliness and organization. Complaints include dirty restrooms, cluttered aisles, and disorganized shelves, which make it difficult to find items. The pandemic has heightened concerns about cleanliness, with customers expecting higher hygiene standards. Addressing these issues by maintaining a clean and well-organized store environment can significantly improve the shopping experience and customer perception of the brand.

- **Theme Name: Product Availability and Stock Issues**
  - **Summary:** Many customers express frustration with the frequent unavailability of products and inconsistent stock levels. Items are often out of stock, and online inventory does not always match in-store availability. This inconsistency leads to inconvenience and dissatisfaction, as customers may need to visit multiple locations to find desired products. Improving inventory management and ensuring better stock levels can enhance customer satisfaction and reduce frustration.

- **Theme Name: Employee Treatment and Morale**
  - **Summary:** Feedback indicates concerns about how employees are treated and their overall morale. Customers perceive that employees are overworked, underpaid, and not adequately supported, which affects their ability to provide good service. There is a call for better wages, benefits, and working conditions to improve employee satisfaction and performance. Addressing these issues can lead to a more motivated workforce and better customer interactions.

These themes provide actionable insights into areas where improvements can be made to enhance the overall customer experience at Walmart.

## Prompt 4: Reduce overlap

In [ ]:
prompt = f"""
You are a qualitative research analyst specializing in Thematic Analysis. Your task is to analyze open-ended customer feedback and systematically extract distinct, recurring themes using qualitative coding techniques.

## **Task**
Conduct a thematic analysis of the provided responses by:
1. Identifying recurring patterns across responses. Only themes that appear multiple times should be included.
2. Assigning a concise, descriptive theme label that represents the core idea.
3. Writing a detailed summary that provides additional context beyond the label, highlighting key aspects and nuances that the name alone may not fully capture.
4. **Focus on identifying the most significant, recurring themes**—avoid identifying too many themes. Ensure each theme is distinct, and only capture the central, most relevant issues.

## **Methodological Guidelines**
- Ensure mutual exclusivity: themes should be distinct and should **not overlap significantly** in meaning.
- Avoid creating too many themes; focus on the most important, recurring issues that truly reflect customer sentiments.
- Theme names should be short and precise, while summaries should provide detailed context, implications, and nuances.
- Maintain actionability: the summary should explain why the theme is relevant and its impact on customer experience.

**Example 1**: Good theme

- Theme Name: Long Checkout Lines
- Summary: A significant number of responses highlight the frustration with long checkout lines due to insufficient staffing. Customers express dissatisfaction with the limited number of open registers, especially during peak hours, leading to extended wait times. This issue is exacerbated by the reliance on self-checkout lanes, which many customers find inconvenient or unsuitable, particularly for those with large purchases or those who prefer human interaction. The lack of cashiers not only affects the speed of service but also the overall shopping experience, as customers feel neglected and undervalued.

This theme is specific and actionable. It highlights a clear operational issue—insufficient staffing at checkout lanes, particularly during busy times. The summary provides enough context to suggest possible solutions (e.g., opening more registers, improving staffing) to address the pain point, making it actionable for process improvements.

**Example 2**: Bad theme

- Theme Name: Customer Service Issues
- Summary: Many responses highlight dissatisfaction with Walmart's customer service, noting poor service quality, lack of empathy, and the prioritization of profit over customer satisfaction. This theme reflects a significant pain point for customers, affecting their overall experience and perception of the brand.

This theme is too general and vague to be actionable. While customer service dissatisfaction is a valid concern, the theme lacks specificity in identifying which aspects of service need improvement (e.g., wait times, employee behavior, training). It also doesn't suggest concrete solutions or areas for operational change. A more focused theme like "Long Checkout Lines" provides a specific, actionable area for improvement, whereas "Customer Service Issues" fails to do so.

## **Output Format**
Each theme should follow this structure:

- Theme Name: (Concise label representing the central idea)
- Summary: (Detailed description providing context, key aspects, and nuances)

## **Question Context**
{question}

## **Dataset**
{formatted_responses}

## **Closing Reminder**
**Remember**: Focus on **distinct**, **non-overlapping themes** that truly reflect the key concerns raised by the customers. Avoid creating too many themes and ensure each theme is meaningful and actionable.
"""

answer = client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[{"role": "user", "content": prompt}],
).choices[0].message.content

display(Markdown(answer))

Based on the provided dataset of customer feedback, I have identified several recurring themes that reflect significant concerns and sentiments expressed by customers. Here are the most prominent themes:

- **Theme Name: Insufficient Checkout Staffing**
  - **Summary**: A prevalent issue among customers is the frustration with long checkout lines due to a lack of open registers and insufficient staffing. Many customers express dissatisfaction with being forced to use self-checkout lanes, which are often crowded and not preferred, especially for those with large purchases or who require assistance. This issue is exacerbated during peak hours and holidays, leading to extended wait times and a negative shopping experience. Customers feel that the lack of human cashiers not only slows down the checkout process but also diminishes the personal interaction they expect during their shopping experience. Addressing this issue by increasing the number of staffed checkout lanes could significantly improve customer satisfaction and reduce wait times.

- **Theme Name: Poor Customer Service**
  - **Summary**: Numerous responses highlight dissatisfaction with the quality of customer service, citing rude and unhelpful staff as a major concern. Customers report feeling neglected and undervalued, with employees often appearing disinterested or dismissive. This lack of engagement and assistance affects the overall shopping experience, as customers struggle to find help when needed. Improving customer service through better training and fostering a more customer-oriented culture could enhance the shopping experience and encourage repeat visits.

- **Theme Name: Store Cleanliness and Organization**
  - **Summary**: Customers frequently mention issues with store cleanliness and organization, noting that stores are often messy, with cluttered aisles and unclean facilities, particularly restrooms. This lack of cleanliness is especially concerning in the context of the ongoing pandemic, where hygiene is of utmost importance. Additionally, the constant rearrangement of store layouts makes it difficult for customers to find items, leading to frustration and a less efficient shopping experience. Enhancing store cleanliness and maintaining a consistent layout could improve customer satisfaction and ease of shopping.

- **Theme Name: Product Availability and Stock Issues**
  - **Summary**: A recurring theme is the frustration with out-of-stock items and inconsistent product availability. Customers often find that items they need are not available, leading them to visit multiple stores or seek alternatives. This issue is compounded by discrepancies between online stock levels and in-store availability, causing further inconvenience. Improving inventory management and ensuring better stock levels could enhance customer satisfaction and reduce the need for customers to shop elsewhere.

- **Theme Name: Self-Checkout Concerns**
  - **Summary**: While some customers appreciate the convenience of self-checkout, many express concerns about the over-reliance on these systems. Customers feel that self-checkout should not replace human cashiers, as it can be challenging for those who are not tech-savvy or have large purchases. Additionally, issues with malfunctioning self-checkout machines and a lack of assistance when problems arise contribute to a frustrating experience. Balancing the use of self-checkout with adequate staffing of traditional registers could address these concerns and improve the overall checkout experience.

These themes reflect the most significant and recurring issues raised by customers, providing actionable insights for improving the customer experience at Walmart.

## Check prompt with different question

In [9]:
question = "What do you LOVE most about Walmart?"
cleaned_responses = clean_responses(df[question])
formatted_responses = "\n".join(cleaned_responses)

In [ ]:
prompt = f"""
You are a qualitative research analyst specializing in Thematic Analysis. Your task is to analyze open-ended customer feedback and systematically extract distinct, recurring themes using qualitative coding techniques.

## **Task**
Conduct a thematic analysis of the provided responses by:
1. Identifying recurring patterns across responses. Only themes that appear multiple times should be included.
2. Assigning a concise, descriptive theme label that represents the core idea.
3. Writing a detailed summary that provides additional context beyond the label, highlighting key aspects and nuances that the name alone may not fully capture.
4. **Focus on identifying the most significant, recurring themes**—avoid identifying too many themes. Ensure each theme is distinct, and only capture the central, most relevant issues.

## **Methodological Guidelines**
- Ensure mutual exclusivity: themes should be distinct and should **not overlap significantly** in meaning.
- Avoid creating too many themes; focus on the most important, recurring issues that truly reflect customer sentiments.
- Theme names should be short and precise, while summaries should provide detailed context, implications, and nuances.
- Maintain actionability: the summary should explain why the theme is relevant and its impact on customer experience.

**Example 1**: Good theme

- Theme Name: Long Checkout Lines
- Summary: A significant number of responses highlight the frustration with long checkout lines due to insufficient staffing. Customers express dissatisfaction with the limited number of open registers, especially during peak hours, leading to extended wait times. This issue is exacerbated by the reliance on self-checkout lanes, which many customers find inconvenient or unsuitable, particularly for those with large purchases or those who prefer human interaction. The lack of cashiers not only affects the speed of service but also the overall shopping experience, as customers feel neglected and undervalued.

This theme is specific and actionable. It highlights a clear operational issue—insufficient staffing at checkout lanes, particularly during busy times. The summary provides enough context to suggest possible solutions (e.g., opening more registers, improving staffing) to address the pain point, making it actionable for process improvements.

**Example 2**: Bad theme

- Theme Name: Customer Service Issues
- Summary: Many responses highlight dissatisfaction with Walmart's customer service, noting poor service quality, lack of empathy, and the prioritization of profit over customer satisfaction. This theme reflects a significant pain point for customers, affecting their overall experience and perception of the brand.

This theme is too general and vague to be actionable. While customer service dissatisfaction is a valid concern, the theme lacks specificity in identifying which aspects of service need improvement (e.g., wait times, employee behavior, training). It also doesn't suggest concrete solutions or areas for operational change. A more focused theme like "Long Checkout Lines" provides a specific, actionable area for improvement, whereas "Customer Service Issues" fails to do so.

## **Output Format**
Each theme should follow this structure:

- Theme Name: (Concise label representing the central idea)
- Summary: (Detailed description providing context, key aspects, and nuances)

## **Question Context**
{question}

## **Dataset**
{formatted_responses}

## **Closing Reminder**
**Remember**: Focus on **distinct**, **non-overlapping themes** that truly reflect the key concerns raised by the customers. Avoid creating too many themes and ensure each theme is meaningful and actionable.
"""

answer = client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[{"role": "user", "content": prompt}],
).choices[0].message.content

display(Markdown(answer))

Based on the provided customer feedback, I have identified the most significant recurring themes. These themes reflect the core aspects that customers appreciate about Walmart. Here are the themes along with their summaries:

- **Theme Name: Low Prices**
  - **Summary**: A predominant theme across the feedback is Walmart's competitive pricing. Customers frequently mention the affordability of products, highlighting that Walmart offers low prices on a wide range of items. This affordability is a key factor in customer satisfaction, as it allows shoppers to purchase more within their budget. The emphasis on low prices is often linked to the perception of value for money, making Walmart a preferred shopping destination for cost-conscious consumers.

- **Theme Name: Product Variety**
  - **Summary**: Many customers appreciate the extensive variety of products available at Walmart. The feedback indicates that Walmart is seen as a "one-stop shop" where customers can find almost everything they need, from groceries and clothing to electronics and home goods. This wide selection is particularly valued for its convenience, as it reduces the need for multiple shopping trips to different stores. The availability of diverse product categories under one roof enhances the overall shopping experience and meets a broad range of customer needs.

- **Theme Name: Convenience**
  - **Summary**: Convenience is a recurring theme, with customers valuing Walmart's accessibility and ease of shopping. The presence of Walmart stores in numerous locations, often close to customers' homes, is a significant advantage. Additionally, the availability of services such as online shopping, curbside pickup, and delivery options further enhances convenience. Customers appreciate the ability to shop quickly and efficiently, whether in-store or online, which fits well with their busy lifestyles.

- **Theme Name: Friendly Employees**
  - **Summary**: Several responses highlight the positive interactions with Walmart employees. Customers note that staff members are generally friendly, helpful, and attentive to their needs. This positive customer service experience contributes to a welcoming shopping environment and enhances customer satisfaction. The presence of approachable and supportive employees is seen as a valuable aspect of the Walmart shopping experience, fostering a sense of community and customer loyalty.

These themes capture the most significant aspects of customer appreciation for Walmart, focusing on affordability, variety, convenience, and customer service. Each theme is distinct and actionable, providing insights into areas where Walmart excels and can continue to build upon to enhance customer satisfaction.

## Structured Output

A quick test of how OpenAI's structured output works. While this is still a beta endpoint, I know other libraries can achieve similar results.

Following OpenAI's documentation here: https://platform.openai.com/docs/guides/structured-outputs?api-mode=chat

In [11]:
from pydantic import BaseModel, Field

class Theme(BaseModel):
    name: str = Field(
        ..., description="A concise label that represents the central idea of the theme"
    )
    summary: str = Field(
        ...,
        description="A brief summary of the theme that provides additional context beyond the name. It should highlight key aspects and nuances that the name alone may not fully capture.",
    )

    def __str__(self) -> str:
        return f"NAME: {self.name}\nSUMMARY: {self.summary}\n\n"

class ThemesResponse(BaseModel):
    themes: list[Theme] = Field(
        ...,
        description="A list of themes extracted from the responses. Each theme should be a concise label and a summary that provides additional context beyond the name.",
    )

In [12]:
answer = client.beta.chat.completions.parse(
    model="gpt-4o",
    temperature=0,
    messages=[{"role": "user", "content": prompt}],
    response_format=ThemesResponse
).choices[0].message.content

themes_response = ThemesResponse.model_validate_json(answer)
for theme in themes_response.themes:
    print(theme)

NAME: Wide Product Variety
SUMMARY: Many customers appreciate Walmart's extensive range of products, which allows them to find almost everything they need in one location. This variety spans groceries, electronics, clothing, home goods, and more, making it a convenient one-stop shop. The availability of diverse items under one roof simplifies shopping and saves time, as customers can complete multiple errands in a single trip.


NAME: Affordable Prices
SUMMARY: A recurring theme is the appreciation for Walmart's low prices, which are often highlighted as competitive and budget-friendly. Customers value the ability to purchase a wide range of products at reasonable costs, which helps them manage their budgets effectively. The perception of good value for money is a significant factor in customer satisfaction and loyalty.


NAME: Convenience and Accessibility
SUMMARY: Customers frequently mention the convenience of Walmart's locations, often noting that stores are nearby and easily acces